In [1]:
import numpy as np
from pathlib import Path
from ranking_utils import scores_better, rankscore_avgtie, greaters, r_names
import pandas as pd

##load data if not present

results_dir = Path(Path.cwd(),'L2R_Continous_NPY') ##
if not results_dir.exists():
    ##load and extract data
    !wget https://cloud.imi.uni-luebeck.de/s/cw2m7XHa87piQY2/download -O L2R_Continous_NPY.zip
    !unzip L2R_Continous_NPY.zip
    
num_labels = 35
num_cases = 38
task_name = 'OASIS' ##discard 2020_MICCAI results

discard_events = ['2020_MICCAI'] #eg. ['2020_MICCAI', '2022_2023_Continous']
discard_teams = ['2021_MICCAI/LapIRN(unsupervised)'] #eg. ['2020_MICCAI/Initial', '2022_2023_Continous/Initial']
                 

In [2]:

results_paths = sorted([x.parent for x in results_dir.glob(f'*/*/{task_name}_dice.npy') if x.is_file()])

###apply filters
results_paths = list(filter(lambda x: x.parent.name not in discard_events and x.parent.name+'/'+x.name not in discard_teams, results_paths))
###move initial to front
initial_index = [i for i, s in enumerate(results_paths) if s.name == 'Initial'][0]
results_paths.insert(0, results_paths.pop(initial_index))

#print(*results_paths, sep = "\n")
events, unique_teams = [x.parent.name for x in results_paths],[x.name for x in results_paths]

N = len(unique_teams)


In [3]:
dice_3 = np.zeros((N,num_cases,num_labels))
hd95_3 = 5*np.ones((N,num_cases,num_labels))
sdlogj_3 = 5*np.ones((N,num_cases))
times_3 = 100*np.ones((N,num_cases))

for i, r_path in enumerate(results_paths):
    team = r_names(r_path.name)
    if Path ( r_path / f'{task_name}_dice.npy').is_file():
        dice = np.load(r_path / f'{task_name}_dice.npy')
        dice[dice != dice] = 0
        dice_3[i,:,:] = dice
    if Path ( r_path / f'{task_name}_hd95.npy').is_file():
        hd95 = np.load(r_path / f'{task_name}_hd95.npy')
        hd95[hd95 != hd95] = 10
        hd95[hd95 == np.Inf] = 10
        hd95_3[i,:,:] = hd95
    if Path ( r_path / f'{task_name}_sd_log_j.npy').is_file() and not team == 'Initial':
        sdlogj = np.load(r_path / f'{task_name}_sd_log_j.npy')
        sdlogj_3[i,:] = sdlogj
    else:
        sdlogj_3[i,:] = 1.5
        
    if Path ( r_path / f'{task_name}_docker_runtimes.npy').is_file() and not team == 'Initial':
        times = np.load(r_path / f'{task_name}_docker_runtimes.npy')
        times_3[i,:] =  times
    else:
        times_3[i,:] = 5000

    # if len (team) < 5:
    #     tab = '\t\t\t\t'
    # elif len (team) < 12:
    #     tab = '\t\t\t'
    # elif len (team) < 24:
    #     tab = '\t\t'
    # else:
    #     tab = '\t'

    # print(i, team, tab + '%0.3f' % dice.mean(), '%0.3f' % hd95.mean(), '%0.3f' % sdlogj_3[i,:].mean(),
    #       '%0.3f' % times_3[i,:].mean())


    


In [4]:
#robustify
dice0 = dice_3[0]  
dice3_30 = np.zeros((N, round((num_cases*num_labels)*.3)))
idx30 = np.argsort(dice0.reshape(-1))[:dice3_30.shape[1]]
for i in range(N):
    dice3_30[i] = dice_3.reshape(N,-1)[i,idx30]


In [5]:
rank_all = np.zeros((N,5))
subset = dice_3.reshape(N,-1)
scores = scores_better(subset, N)
rank_dice3a = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,0] = rank_dice3a
subset = dice3_30.reshape(N,-1)
scores = scores_better(subset, N)
rank_dice3b = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,1] = rank_dice3b
subset = hd95_3.reshape(N,-1)
scores = scores_better(-subset, N)
rank_hd3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,2] = rank_hd3
subset = sdlogj_3.reshape(N,-1)
scores = scores_better(-subset, N)
rank_jac3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,3] = rank_jac3
subset = times_3.reshape(N,-1)
scores = scores_better(-subset, N)
rank_time3 = rankscore_avgtie(-scores.astype('int64'))
rank_all[:,4] = rank_time3

In [6]:
rank3 = np.power(np.prod(rank_all[:,:],axis=1),.2)
all_rank = np.power(np.prod(rank_all,axis=1),1/5)

In [7]:
df = pd.DataFrame(columns=['rank','team','Dice','Dice30','HD95',' sdLogJ','rank'])

rank3_withoutRUNTIME = np.power(np.prod(rank_all[:,:4],axis=1),.25)
all_rank_withoutRUNTIME = np.power(np.prod(rank_all[:,:4],axis=1),1/4)
idx2 = np.argsort(-all_rank_withoutRUNTIME)


print('WITHOUT RUNTIME')
print('#','team','\t\t\t','Dice\t','Dice30\t','HD95\t',' sdLogJ','rank')


for i in range(N):
    team = r_names(unique_teams[idx2[i]])
    new = ''
    if events[idx2[i]] == '2023_MICCAI':
        new = '> '
    tab = '\t\t\t'
    d0 = 5
    d1 = 6
    if(len(team)>=d0):
        tab = '\t\t'
    if(len(team)>d0+d1):
        tab = '\t'
    
    if(len(team)>=d0+d1*2):
        tab = ''
    #if(len(team)>=d0+d1*3):
    #    tab = ''
    print(i+1,new + team,tab,'%0.3f'%dice_3[idx2[i]].mean(),'\t','%0.3f'%dice3_30[idx2[i]].mean(),'\t','%0.3f'%hd95_3[idx2[i]].mean(),' ','%0.3f'%sdlogj_3[idx2[i]].mean(),' ','%0.3f'%rank3_withoutRUNTIME[idx2[i]])
    df.loc[i] = [i+1,new + team,'%0.3f'%dice_3[idx2[i]].mean(),'%0.3f'%dice3_30[idx2[i]].mean(),'%0.3f'%hd95_3[idx2[i]].mean(),'%0.3f'%sdlogj_3[idx2[i]].mean(),'%0.3f'%rank3_withoutRUNTIME[idx2[i]]]


WITHOUT RUNTIME
# team 			 Dice	 Dice30	 HD95	  sdLogJ rank
1 LapIRN 		 0.822 	 0.664 	 1.668   0.071   0.814
2 JunyuChen 		 0.829 	 0.675 	 1.580   0.093   0.791
3 ConvexAdam 		 0.811 	 0.641 	 1.629   0.070   0.763
4 > LapIRNv2 		 0.836 	 0.690 	 1.562   0.124   0.750
5 BinDuan 		 0.820 	 0.659 	 1.666   0.085   0.716
6 IWM 			 0.794 	 0.608 	 1.844   0.051   0.669
7 KunTang 		 0.823 	 0.666 	 1.658   0.120   0.666
8 YihaoLiu 		 0.834 	 0.681 	 1.658   0.234   0.651
9 Driver 		 0.799 	 0.620 	 1.775   0.079   0.646
10 3Idiots 		 0.803 	 0.632 	 1.819   0.082   0.622
11 PIMed 		 0.783 	 0.577 	 1.863   0.065   0.619
12 Winter 		 0.770 	 0.571 	 2.157   0.080   0.449
13 MEVIS 		 0.769 	 0.569 	 2.094   0.071   0.448
14 corrField 		 0.766 	 0.567 	 2.022   0.095   0.383
15 Multi-Brain 		 0.776 	 0.594 	 1.921   0.569   0.348
16 Thorley 		 0.769 	 0.597 	 2.213   0.312   0.332
17 Imperial 		 0.760 	 0.573 	 2.433   0.189   0.309
18 LaTIM 		 0.739 	 0.515 	 2.313   0.079   0.308
19 NiftyR

In [8]:
idx2 = np.argsort(-all_rank)
print('WITH RUNTIME')
print('#','team','\t\t\t','Dice\t','Dice30\t','HD95\t ','sdLogJ ','runtime\t','rank')
for i in range(N):
    team = r_names(unique_teams[idx2[i]])
    new = ''
    if events[idx2[i]] == '2023_MICCAI':
        new = '> '
    tab = '\t\t\t'
    d0 = 5
    d1 = 6
    if(len(team)>=d0):
        tab = '\t\t'
    if(len(team)>d0+d1):
        tab = '\t'
    if(len(team)>=d0+d1*2):
        tab = ''
    #if(len(team)>=d0+d1*3):
    #    tab = ''
    print(i+1,new+team,tab,'%0.3f'%dice_3[idx2[i]].mean(),'\t','%0.3f'%dice3_30[idx2[i]].mean(),'\t','%0.3f'%hd95_3[idx2[i]].mean(),' ','%0.3f'%sdlogj_3[idx2[i]].mean(),' ','%0.4f'%times_3[idx2[i]].mean(),'\t','%0.3f'%rank3_withoutRUNTIME[idx2[i]])


WITH RUNTIME
# team 			 Dice	 Dice30	 HD95	  sdLogJ  runtime	 rank
1 LapIRN 		 0.822 	 0.664 	 1.668   0.071   1.2057 	 0.814
2 ConvexAdam 		 0.811 	 0.641 	 1.629   0.070   3.0998 	 0.763
3 IWM 			 0.794 	 0.608 	 1.844   0.051   2.5548 	 0.669
4 Driver 		 0.799 	 0.620 	 1.775   0.079   2.0188 	 0.646
5 3Idiots 		 0.803 	 0.632 	 1.819   0.082   1.4592 	 0.622
6 JunyuChen 		 0.829 	 0.675 	 1.580   0.093   5000.0000 	 0.791
7 PIMed 		 0.783 	 0.577 	 1.863   0.065   3.4722 	 0.619
8 > LapIRNv2 		 0.836 	 0.690 	 1.562   0.124   5000.0000 	 0.750
9 BinDuan 		 0.820 	 0.659 	 1.666   0.085   5000.0000 	 0.716
10 KunTang 		 0.823 	 0.666 	 1.658   0.120   5000.0000 	 0.666
11 YihaoLiu 		 0.834 	 0.681 	 1.658   0.234   5000.0000 	 0.651
12 Winter 		 0.770 	 0.571 	 2.157   0.080   2.5558 	 0.449
13 MEVIS 		 0.769 	 0.569 	 2.094   0.071   10.3557 	 0.448
14 corrField 		 0.766 	 0.567 	 2.022   0.095   13.9658 	 0.383
15 Imperial 		 0.760 	 0.573 	 2.433   0.189   2610.6623 	 0.309
16 Ni

In [9]:
print(df)

   rank         team   Dice Dice30   HD95  sdLogJ   rank
0     1       LapIRN  0.822  0.664  1.668   0.071  0.814
1     2    JunyuChen  0.829  0.675  1.580   0.093  0.791
2     3   ConvexAdam  0.811  0.641  1.629   0.070  0.763
3     4   > LapIRNv2  0.836  0.690  1.562   0.124  0.750
4     5      BinDuan  0.820  0.659  1.666   0.085  0.716
5     6          IWM  0.794  0.608  1.844   0.051  0.669
6     7      KunTang  0.823  0.666  1.658   0.120  0.666
7     8     YihaoLiu  0.834  0.681  1.658   0.234  0.651
8     9       Driver  0.799  0.620  1.775   0.079  0.646
9    10      3Idiots  0.803  0.632  1.819   0.082  0.622
10   11        PIMed  0.783  0.577  1.863   0.065  0.619
11   12       Winter  0.770  0.571  2.157   0.080  0.449
12   13        MEVIS  0.769  0.569  2.094   0.071  0.448
13   14    corrField  0.766  0.567  2.022   0.095  0.383
14   15  Multi-Brain  0.776  0.594  1.921   0.569  0.348
15   16      Thorley  0.769  0.597  2.213   0.312  0.332
16   17     Imperial  0.760  0.

In [10]:
##save to csv
df.to_csv('OASIS_ranking.csv', index=False)
